In [1]:
import numpy as np 
import pandas as pd 
import requests 
import json
import time
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set()

In [2]:
def getEricRecords(search, fields = None, start=0, rows=200):
    url = 'https://api.ies.ed.gov/eric/?'
    url = url + 'search=' + search + '&rows=' + str(rows) + '&format=json&start=' + str(start) 
    if(fields):
        url = url + '&fields=' + ', '.join(fields)
    responseJson = requests.get(url).json() 
    return pd.DataFrame(responseJson)

In [6]:
def getRecordCount(search):
    dataFrame = getEricRecords(search)
    totalRecords = dataFrame.loc['numFound'][0]
    print('Search', search, 'returned', '{:,}'.format(totalRecords), 'records') 
    return totalRecords

In [7]:
def cleanElementsUsingList(x): 
    if(not isinstance(x, list)):
        return x
    if(not x or (len(x) == 1 and x[0] == '')):
        return None 
    return ', '.join(x)

In [8]:
def getAllEricRecords(search, fields = None, cleanElements = True): 
    startTime = time.time()
    nextFirstRecord = 0
    numRecordsReturnedEachApiCall = 200
    totalRecords = getRecordCount(search) 
    if(totalRecords == 0):
        print ('Search', search, 'has no results') 
        return []
    while(nextFirstRecord < totalRecords):
        dataFrame = getEricRecords(search, fields, nextFirstRecord) 
        if(nextFirstRecord == 0):
            records = pd.DataFrame(dataFrame.loc['docs'][0]) 
        else:
            records = pd.concat([records, pd.DataFrame(dataFrame.loc['docs'][0])], sort=False, ignore_index=True) 
        nextFirstRecord += numRecordsReturnedEachApiCall
    print('took', '{:,.1f}'.format(time.time() - startTime), 'seconds')
    return records.applymap(cleanElementsUsingList) if cleanElements else records

In [9]:
search = 'subject:autism AND subject:”teaching methods” AND publicationdateyear:2019'
records = getAllEricRecords(search)
records

Search subject:autism AND subject:”teaching methods” AND publicationdateyear:2019 returned 88 records
took 2.4 seconds


,id,title,author,description,subject,publicationtype,publicationdateyear,language,issn,publisher,peerreviewed,isbn
0,EJ1215413,Teaching Written Expression to Students with A...,"Pennington, Robert C., Carpenter, Megan",Written expression is critical to the full par...,"Autism, Pervasive Developmental Disorders, Wri...","Journal Articles, Reports - Descriptive",2019,English,ISSN-0271-8294,Lippincott Williams & Wilkins. Available from:...,T,NaN
1,EJ1235576,Teaching Mands for Information Using &quot;Why...,"Valentino, Amber L., Fu, Sherrene Brice, Padov...",Mands for information (MFIs) play an important...,"Verbal Operant Conditioning, Autism, Teaching ...","Journal Articles, Reports - Research",2019,English,ISSN-0889-9401,Springer. Available from: Springer Nature. 233...,T,NaN
2,EJ1224583,A Review of Language Development Protocols for...,"Ackley, Meleah, Subramanian, Jennifer W., Moor...",The present review sought to examine contempor...,"Autism, Applied Behavior Analysis, Evaluation ...","Journal Articles, Reports - Evaluative",2019,English,ISSN-1053-0819,Springer. Available from: Springer Nature. 233...,T,NaN
3,EJ1215335,Increasing Play Complexity in a Young Child wi...,"Barton, Erin E., Gossett, Stephanie, Waters, M...",Young children with autism spectrum disorder (...,"Preschool Children, Autism, Pervasive Developm...","Journal Articles, Reports - Research",2019,English,ISSN-1088-3576,SAGE Publications and Hammill Institute on Dis...,T,NaN
4,EJ1230200,Appropriate Inclusive Practices for Children w...,"Sawchuk, Amber","Children with autism have unique needs, and pr...","Inclusion, Autism, Teacher Collaboration, Educ...","Journal Articles, Reports - Descriptive",2019,English,ISSN-,"Brandon University. 270 18th Street, Brandon, ...",T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
83,ED598631,Differentiation for Gifted Learners: Going bey...,"Heacox, Diane, Cash, Richard M.",With increasing numbers of students receiving ...,"Academically Gifted, Individualized Instructio...","Books, Guides - General",2019,English,NaN,Free Spirit Publishing. 6325 Sandburg Road Sui...,F,978-1-63198-432-7
84,ED593866,Barriers to Educator Implementation of a Class...,"Wilson, Kaitlyn P., Landa, Rebecca J.",There is considerable need for adoptable evide...,"Preschool Children, Evidence Based Practice, A...","Journal Articles, Reports - Research",2019,English,EISSN-,NaN,T,NaN
85,ED602800,Promise in Infant-Toddler Care and Education. ...,"Boldt, Gail, Ed.",Much of the policy-and practice-focused resear...,"Educational Quality, Infants, Toddlers, Early ...",Collected Works - General,2019,English,NaN,Bank Street College of Education. 610 West 112...,T,NaN
86,ED593669,Educating Young Children with and without Exce...,"Obiakor, Festus E., Ed., Banks, Tachelle, Ed.,...",Early childhood education (ECE) has always bee...,"Early Childhood Education, Disabilities, Devel...","Books, Collected Works - General",2019,English,NaN,"IAP - Information Age Publishing, Inc. P.O. Bo...",F,978-1-64113-594-8
